# Investment strategy weight by premium/rebate

In [ ]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from yahoofinancials import YahooFinancials
import lxml.html as lh
import lxml.etree as etree
from selenium import webdriver
from requests.utils import quote
import time
import re
import math
import matplotlib.pyplot as plt

### Read configuration file

In [ ]:
# Read df and set nan to empty string
stocks_df = pd.read_csv('ib_stocks_stockholm.csv')
stocks_df.fillna('', inplace=True)

# List of stocks
stocks_list = stocks_df['Ticker'].tolist()

# Get url-ending list
url_ending_list = stocks_df['URL_PATH'].tolist()

# Get element-name list
element_names = stocks_df['ELEMENT_NAME'].tolist()

# Comply with format of yahoo finance for swedish stocks
stocks_df['Ticker'] = stocks_df['Ticker'].str.replace(' ','-')
stocks_df['Ticker'] = stocks_df['Ticker'].astype(str) + '.ST'
stocks_list_yf = stocks_df['Ticker'].tolist()

### Read current stock price from yahoo financials

In [ ]:
yahoo_financials_ib = YahooFinancials(stocks_list_yf)
current_price = yahoo_financials_ib.get_current_price()
current_price

### Crawl pages to read reported and computed rebate/premium values

In [ ]:
driver = webdriver.Firefox()
driver.set_page_load_timeout(10)

base_url='https://ibindex.se/ibi/#/'
html_pages = {}
for index, url_ending in enumerate(url_ending_list):
    driver.get(f'{base_url}{quote(url_ending)}')
    driver.refresh()
    time.sleep(4)
    html = driver.page_source
    html_pages[stocks_list[index]] = html
driver.quit()

### Create data structure for calculated and reported rebate/premiums sorted by stocks

In [ ]:
rebate_premium_dict = {}
regexp_number = '(\d{1,2}.\d{1,2})'
regexp_rebate_premium = '(rabatt|premie)'

for index, symbol in enumerate(stocks_list):
    doc = lh.fromstring(html_pages[symbol])
    if(element_names[index] != ''):
        rebate_premium_list = doc.xpath(f'//{element_names[index]}//span')
    else:
        rebate_premium_list = doc.xpath('//app-company-current-rebate-premium//span')
    vals = [el.text for el in rebate_premium_list]
    if(len(vals) == 6):
        val1 = float(re.search(regexp_number, vals[1]).group(1))
        val2 = float(re.search(regexp_number, vals[4]).group(1))
        valtype1 = re.search(regexp_rebate_premium, vals[2]).group(1)
        valtype2 = re.search(regexp_rebate_premium, vals[5]).group(1)
    else:
        val1 = 0
        val2 = 0
        valtype1 = 'premium'
        valtype2 = 'premium'
          
    computed_dict = {
        'val': val1,
        'type': valtype1
    }
    reported_dict = {
        'val': val2,
        'type': valtype2
    }
    rebate_premium_dict[symbol] = {'computed': computed_dict, 'reported': reported_dict}

### Create dataframe

In [ ]:
my_columns = ['Ticker', 'Price', 'Reported Val', 'Reported Type', 'Reported Weight','Reported Position', 'Computed Val', 'Computed Type', 'Computed Weight','Computed Position', 'Rep Buy', 'Comp Buy']
final_dataframe = pd.DataFrame(columns = my_columns)

for idx, symbol_yf in enumerate(stocks_list_yf):
    symbol = stocks_list[idx]
    computed_val = rebate_premium_dict[symbol]['computed']['val']
    computed_type = rebate_premium_dict[symbol]['computed']['type']
    reported_val = rebate_premium_dict[symbol]['reported']['val']
    reported_type = rebate_premium_dict[symbol]['reported']['type']
    series_list = [symbol, current_price[symbol_yf], reported_val, reported_type, 'N/A', 'N/A', computed_val, computed_type, 'N/A', 'N/A', 'N/A', 'N/A']
    panda_series = pd.Series(series_list, index = my_columns)
    final_dataframe = final_dataframe.append(panda_series, ignore_index=True)

reported_sum = 0
computed_sum = 0
for index, row in final_dataframe.iterrows():
    if(row['Reported Type'] == 'rabatt'):
        reported_sum += row['Reported Val']
    if(row['Computed Type'] == 'rabatt'):
        computed_sum += row['Computed Val']

final_dataframe['Reported Weight'] = final_dataframe['Reported Val']/reported_sum
final_dataframe.loc[final_dataframe['Reported Type'] == 'premie', 'Reported Weight'] = 0 
final_dataframe['Computed Weight'] = final_dataframe['Computed Val']/computed_sum
final_dataframe.loc[final_dataframe['Computed Type'] == 'premie', 'Computed Weight'] = 0

### Input portfolio size

In [ ]:
portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")

### Compute position to the different stocks

In [ ]:
final_dataframe['Reported Position'] = final_dataframe['Reported Weight']*val
final_dataframe['Computed Position'] = final_dataframe['Computed Weight']*val

### Create final dataframe

In [ ]:
def styler(df):
    color = 'color: {}'.format
    mask = pd.concat([df['Reported Type'].gt('premium')] * df.shape[1], axis=1)
    style = np.where(mask, color('green'), color('red'))
    return style

# Buy
final_dataframe['Comp Buy'] = final_dataframe['Computed Position']/final_dataframe['Price']
final_dataframe['Comp Buy'] = final_dataframe['Comp Buy'].apply(np.floor)

final_dataframe['Rep Buy'] = final_dataframe['Reported Position']/final_dataframe['Price']
final_dataframe['Rep Buy'] = final_dataframe['Rep Buy'].apply(np.floor)
styled = final_dataframe.style.apply(styler, axis=None)
styled


### Plot distribution of stocks to buy - Reported Rebate

In [ ]:
labels = []
sizes = []
stocks_count = 0
for index, row in final_dataframe.iterrows():
    if(row['Rep Buy'] != 0):
        labels.append(row['Ticker'])
        sizes.append(row['Rep Buy']*row['Price'])
        stocks_count += row['Rep Buy']

fig, ax = plt.subplots(figsize=(15, 8), subplot_kw=dict(aspect="equal"))
#ax.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

wedges, texts = ax.pie(sizes, wedgeprops=dict(width=0.5), startangle=-40)

bbox_props = dict(boxstyle="square,pad=0.3", fc="w", ec="k", lw=0.72)
kw = dict(arrowprops=dict(arrowstyle="-"),
          bbox=bbox_props, zorder=0, va="center")

for i, p in enumerate(wedges):
    ang = (p.theta2 - p.theta1)/2. + p.theta1
    y = np.sin(np.deg2rad(ang))
    x = np.cos(np.deg2rad(ang))
    horizontalalignment = {-1: "right", 1: "left"}[int(np.sign(x))]
    connectionstyle = "angle,angleA=0,angleB={}".format(ang)
    kw["arrowprops"].update({"connectionstyle": connectionstyle})
    ax.annotate(sizes[i], xy=(x, y), xytext=(1.35*np.sign(x), 1.4*y),
                horizontalalignment=horizontalalignment, **kw)

ax.legend(wedges, labels, title="Stocks", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))
ax.set_title('Buy proportions')

plt.show()

data = {'Budget': [val], 'Buy Sum': [sum(sizes)], 'Money left': [val-sum(sizes)], 'Total shares to buy': [stocks_count], 'Company count': [len(sizes)]}
print(pd.DataFrame.from_dict(data))